In [1]:
%load_ext autoreload
%autoreload 2

In [80]:
import os 
import sys
import time

project_dir_path = '/home/onoue/ws/lukasiewicz_1'
sys.path.append(project_dir_path)

for path in sys.path:
    print(path)

from src.setup_problem import Setup
from src.misc import visualize_result

import cvxpy as cp

from src.misc import *

/home/onoue/ws/lukasiewicz_1/notebooks/pima_indian_diabetes
/usr/lib/python310.zip
/usr/lib/python3.10
/usr/lib/python3.10/lib-dynload

/home/onoue/ws/lukasiewicz_1/myenv/lib/python3.10/site-packages
/home/onoue/ws/lukasiewicz_1
/home/onoue/ws/lukasiewicz_1


# データの準備

In [66]:
train_index = pd.read_csv("/home/onoue/ws/lukasiewicz_1/inputs/pima_indian_diabetes/train_index.csv", index_col=0)["0"].to_list()
# display(train_index)

test_index = pd.read_csv("/home/onoue/ws/lukasiewicz_1/inputs/pima_indian_diabetes/test_index.csv", index_col=0)["0"].to_list()
# display(test_index)

In [79]:
data = pd.read_csv('./data/diabetes_discretized.csv', index_col=0).loc[train_index, :]

outcome = data['Outcome']
features = data.drop(['Outcome'], axis=1)
feature_names = list(features.columns)


data_dir_path = os.path.join(project_dir_path, 'inputs/pima_indian_diabetes/train')

if not os.path.exists(data_dir_path):
    os.mkdir(data_dir_path)


df = features.copy()
df['target'] = outcome.replace(0, -1)
display(df)

file_name = "L_" + "Outcome" + '.csv'
file_path = os.path.join(data_dir_path, file_name)
df.to_csv(file_path)


for feature_name in feature_names:
    df = features.copy()
    df['target'] = df[feature_name].replace(0, -1)
    # display(df)

    file_name = "L_" + feature_name + '.csv'
    file_path = os.path.join(data_dir_path, file_name)
    df.to_csv(file_path)


data_num = 15
data_dim = 21

arr_U = np.random.randint(2, size=(data_num, data_dim))
df_U = pd.DataFrame(arr_U)
df_U.to_csv(os.path.join(data_dir_path, 'U.csv'))

,Pregnancies_Low,Pregnancies_Medium,Pregnancies_High,Glucose_Low,Glucose_Medium,Glucose_High,BloodPressure_Low,BloodPressure_Medium,BloodPressure_High,SkinThickness_Low,...,BMI_Low,BMI_Medium,BMI_High,DiabetesPedigreeFunction_Low,DiabetesPedigreeFunction_Medium,DiabetesPedigreeFunction_High,Age_Low,Age_Medium,Age_High,target
275,1,0,0,0,1,0,0,1,0,0,...,0,0,1,0,1,0,1,0,0,-1
555,0,1,0,0,1,0,0,1,0,0,...,1,0,0,1,0,0,0,1,0,-1
699,1,0,0,0,1,0,0,1,0,1,...,0,0,1,0,0,1,1,0,0,-1
73,1,0,0,0,1,0,0,0,1,1,...,0,1,0,1,0,0,1,0,0,-1
170,0,1,0,0,1,0,0,1,0,1,...,0,1,0,1,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,1,0,0,0,1,0,0,1,0,0,...,0,1,0,1,0,0,1,0,0,-1
128,1,0,0,0,1,0,0,0,1,0,...,0,1,0,1,0,0,0,1,0,1
315,1,0,0,0,1,0,0,1,0,0,...,0,1,0,1,0,0,1,0,0,-1
505,0,0,1,1,0,0,0,1,0,1,...,0,1,0,1,0,0,0,1,0,-1


In [77]:
data = pd.read_csv('./data/diabetes_discretized.csv', index_col=0).loc[test_index, :]

outcome = data['Outcome']
features = data.drop(['Outcome'], axis=1)
feature_names = list(features.columns)


data_dir_path = os.path.join(project_dir_path, 'inputs/pima_indian_diabetes/test')

if not os.path.exists(data_dir_path):
    os.mkdir(data_dir_path)


df = features.copy()
df['target'] = outcome.replace(0, -1)
display(df)

file_name = "L_" + "Outcome" + '.csv'
file_path = os.path.join(data_dir_path, file_name)
df.to_csv(file_path)


# for feature_name in feature_names:
#     df = features.copy()
#     df['target'] = df[feature_name].replace(0, -1)
#     # display(df)

#     file_name = "L_" + feature_name + '.csv'
#     file_path = os.path.join(data_dir_path, file_name)
#     df.to_csv(file_path)

,Pregnancies_Low,Pregnancies_Medium,Pregnancies_High,Glucose_Low,Glucose_Medium,Glucose_High,BloodPressure_Low,BloodPressure_Medium,BloodPressure_High,SkinThickness_Low,...,BMI_Low,BMI_Medium,BMI_High,DiabetesPedigreeFunction_Low,DiabetesPedigreeFunction_Medium,DiabetesPedigreeFunction_High,Age_Low,Age_Medium,Age_High,target
419,1,0,0,0,1,0,0,1,0,0,...,1,0,0,1,0,0,1,0,0,1
186,0,1,0,0,0,1,0,1,0,0,...,0,1,0,0,1,0,0,0,1,1
556,1,0,0,0,1,0,0,1,0,0,...,0,1,0,1,0,0,1,0,0,-1
738,1,0,0,0,1,0,1,0,0,1,...,0,1,0,0,1,0,1,0,0,-1
320,1,0,0,0,1,0,1,0,0,1,...,1,0,0,0,1,0,1,0,0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
626,1,0,0,0,1,0,0,1,0,1,...,1,0,0,1,0,0,1,0,0,-1
487,1,0,0,0,0,1,0,1,0,0,...,0,0,1,0,0,1,0,0,1,-1
638,0,1,0,0,1,0,0,1,0,0,...,0,0,1,0,0,1,1,0,0,1
248,0,0,1,0,1,0,0,1,0,0,...,0,1,0,1,0,0,1,0,0,-1


In [81]:
data_dir_path = os.path.join(project_dir_path, 'inputs/pima_indian_diabetes')

train_data_dir_path = os.path.join(data_dir_path, "train")

file_list = os.listdir(train_data_dir_path)

L_files = [filename.split('.csv')[0] for filename in file_list 
           if filename.startswith('L') and filename.endswith('.csv')]

U_files = [filename.split('.csv')[0] for filename in file_list 
           if filename.startswith('U') and filename.endswith('.csv')]

file_names_dict = {
    'supervised': L_files,
    'unsupervised': U_files,
    'rule': ['rules']
}

problem_instance = Setup(train_data_dir_path, file_names_dict)
objective, constraints = problem_instance.main(c1=10, c2=10)

start_time = time.time()
problem = cp.Problem(objective, constraints)
result = problem.solve(verbose=True)
end_time = time.time()
print()
print(f'学習時間: {end_time - start_time} 秒')

Loading data ...
Done in 0.1338198184967041 seconds! 

Loading rules ...
Done in 0.00029349327087402344 seconds! 

Identifying predicates ...
Done in 0.0013189315795898438 seconds! 

Constructing objective function ...
Done in 3.8509159088134766 seconds! 

Constructing constraints ...
Done in 17.18113136291504 seconds! 

All done


/home/onoue/ws/lukasiewicz_1/myenv/lib/python3.10/site-packages/cvxpy/problems/problem.py:157: UserWarning: Objective contains too many subexpressions. Consider vectorizing your CVXPY code to speed up compilation.
  warnings.warn("Objective contains too many subexpressions. "


                                     CVXPY                                     
                                     v1.3.2                                    
(CVXPY) Nov 25 11:59:48 AM: Your problem has 12322 variables, 37416 constraints, and 0 parameters.
(CVXPY) Nov 25 11:59:52 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Nov 25 11:59:52 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Nov 25 11:59:52 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Nov 25 11:59:57 AM: Compiling problem (target solver=ECOS).
(CVXPY) Nov 25 11:59:57 AM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuf

In [88]:
test_data_dir_path = os.path.join(data_dir_path, "test")
file_path = os.path.join(test_data_dir_path, "L_Outcome.csv")
test_df = pd.read_csv(file_path, index_col=0)
display(test_df.head())

test_data = {
    'Outcome': np.array(test_df)
}

test_data

,Pregnancies_Low,Pregnancies_Medium,Pregnancies_High,Glucose_Low,Glucose_Medium,Glucose_High,BloodPressure_Low,BloodPressure_Medium,BloodPressure_High,SkinThickness_Low,...,BMI_Low,BMI_Medium,BMI_High,DiabetesPedigreeFunction_Low,DiabetesPedigreeFunction_Medium,DiabetesPedigreeFunction_High,Age_Low,Age_Medium,Age_High,target
419,1,0,0,0,1,0,0,1,0,0,...,1,0,0,1,0,0,1,0,0,1
186,0,1,0,0,0,1,0,1,0,0,...,0,1,0,0,1,0,0,0,1,1
556,1,0,0,0,1,0,0,1,0,0,...,0,1,0,1,0,0,1,0,0,-1
738,1,0,0,0,1,0,1,0,0,1,...,0,1,0,0,1,0,1,0,0,-1
320,1,0,0,0,1,0,1,0,0,1,...,1,0,0,0,1,0,1,0,0,-1


{'Outcome': array([[ 1,  0,  0, ...,  0,  0,  1],
        [ 0,  1,  0, ...,  0,  1,  1],
        [ 1,  0,  0, ...,  0,  0, -1],
        ...,
        [ 0,  1,  0, ...,  0,  0,  1],
        [ 0,  0,  1, ...,  0,  0, -1],
        [ 0,  0,  1, ...,  1,  0,  1]])}

In [89]:
p_dict = problem_instance.predicates_dict
selected_predicates = ['Outcome']
selected_p_dict = {key: value for key, value in p_dict.items() if key in selected_predicates}
selected_p_dict

{'Outcome': <src.misc.Predicate at 0x7f10afc3e530>}

In [90]:
res_dict = test_trained_predicate(selected_p_dict, test_data)

104
Accuracy of Outcome: 0.7761194029850746


In [94]:
display(pd.DataFrame(res_dict['Outcome']))

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,2.046752e-12,0.0
1,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.000000e+00,1.0
2,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,-1.0,1.320880e-11,1.0
3,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,-1.0,2.215794e-11,1.0
4,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,-1.0,1.099587e-11,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,-1.0,1.080108e-12,1.0
130,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,-1.0,1.000000e+00,0.0
131,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,2.894288e-11,0.0
132,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,-1.0,2.817105e-11,1.0


In [8]:
data_winston = pd.read_csv('./../winston/data/data.csv', index_col=0)
labels_winston = pd.read_csv('./../winston/data/labels.csv', index_col=0)

display(data_winston)
display(labels_winston)

,folder,name,width,height,depth,R,G,B
0,penguin,8065,1200,1600,3,0.486640,0.440085,0.342231
1,penguin,8610,468,500,3,0.427333,0.406967,0.376660
2,penguin,8445,500,375,3,0.509321,0.512104,0.398580
3,penguin,1188,85,62,3,0.581645,0.572609,0.549550
4,penguin,8237,375,500,3,0.505475,0.517679,0.422280
...,...,...,...,...,...,...,...,...
6495,zebra,2311,333,500,3,0.618658,0.583356,0.521907
6496,zebra,6650,379,500,3,0.506803,0.401962,0.288625
6497,zebra,2627,500,375,3,0.517260,0.509384,0.498286
6498,zebra,7258,375,500,3,0.470308,0.439706,0.373938


,albatross,cheetah,ostrich,penguin,zebra
hair(x),NaN,1.0,NaN,NaN,1.0
mammal(x),NaN,1.0,NaN,NaN,1.0
milk(x),NaN,1.0,NaN,NaN,NaN
feather(x),1.0,NaN,1.0,1.0,NaN
bird(x),1.0,NaN,1.0,1.0,NaN
layeggs(x),1.0,NaN,1.0,1.0,NaN
meat(x),NaN,1.0,NaN,NaN,NaN
carnivore(x),NaN,1.0,NaN,NaN,NaN
pointedteeth(x),NaN,1.0,NaN,NaN,NaN
claws(x),NaN,1.0,NaN,NaN,NaN
